In [1]:
import sys
print(sys.path)



['C:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python310', 'n:\\Thesis 2\\Code\\myenv', '', 'n:\\Thesis 2\\Code\\myenv\\lib\\site-packages', 'n:\\Thesis 2\\Code\\myenv\\lib\\site-packages\\win32', 'n:\\Thesis 2\\Code\\myenv\\lib\\site-packages\\win32\\lib', 'n:\\Thesis 2\\Code\\myenv\\lib\\site-packages\\Pythonwin']


In [1]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from tensorflow.keras import utils

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

import seaborn as sns
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)


In [3]:
spotify_df = pd.read_csv('Songs folder/Spotify_Tracks.csv')


In [4]:
spotify_df.isna().sum()


id                   0
name                71
popularity           0
duration_ms          0
explicit             0
artists              0
id_artists           0
release_date         0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
time_signature       0
dtype: int64

In [5]:
spotify_df.head()


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [6]:
mood_prep = spotify_df[['duration_ms', 'danceability', 'acousticness', 'energy', 'instrumentalness',
       'liveness', 'valence', 'loudness', 'speechiness', 'tempo']]

In [7]:
col_features = mood_prep.columns[:]
col_features

Index(['duration_ms', 'danceability', 'acousticness', 'energy',
       'instrumentalness', 'liveness', 'valence', 'loudness', 'speechiness',
       'tempo'],
      dtype='object')

In [8]:
mood_trans = MinMaxScaler().fit_transform(mood_prep[col_features])
mood_trans_np = np.array(mood_prep[col_features])

In [9]:
mood_trans_np[10011]

array([ 2.01107e+05,  5.39000e-01,  9.95000e-01,  2.84000e-01,
        8.94000e-01,  3.20000e-01,  6.19000e-01, -1.42710e+01,
        4.38000e-02,  1.13618e+02])

In [10]:
df = pd.read_csv('Songs folder/data_moods.csv')

In [11]:
cl_features = df.columns[6:-3]
X= MinMaxScaler().fit_transform(df[cl_features])
X2 = np.array(df[cl_features])
Y = df['mood']

In [12]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(Y)


dummy_y = utils.to_categorical(encoded_y)

X_train,X_test,Y_train,Y_test = train_test_split(X,encoded_y,test_size=0.2,random_state=15)

target = pd.DataFrame({'mood':df['mood'].tolist(),'encode':encoded_y}).drop_duplicates().sort_values(['encode'],ascending=True)
target

,mood,encode
5,Calm,0
4,Energetic,1
0,Happy,2
1,Sad,3


In [13]:
def base_model():
    model = Sequential()
    model.add(Dense(8,input_dim=10,activation='relu'))
    model.add(Dense(4,activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',
                 metrics=['accuracy'])
    return model

In [14]:
#Configure the model
estimator = KerasClassifier(build_fn=base_model,epochs=300,batch_size=200,verbose=0)

In [15]:
# # Check the number of unique classes in the target variable
# print(Y.unique())  # or np.unique(Y)

# # Convert the target variable to one-hot encoding
# encoded_y = utils.to_categorical(encoded_y)

# # Ensure the number of neurons in the output layer matches the number of unique classes
# num_classes = len(np.unique(Y))
# # Modify your model architecture to have num_classes neurons in the output layer

# # Double-check the shape of input data and target variable
# print(X.shape)
# print(encoded_y.shape)




In [16]:
# Check the shape of the original target variable
print("Shape of original encoded_y:", encoded_y.shape)

# Reshape the target variable to ensure it has the same number of samples as X
encoded_y_1d = np.ravel(encoded_y)
encoded_y_1d = encoded_y_1d[:686]

# Print the shape of the reshaped target variable
print("Shape of reshaped encoded_y_1d:", encoded_y_1d.shape)






# Evaluate the model using KFold cross validation
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, encoded_y_1d, cv=kfold)

# Print the results
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Shape of original encoded_y: (686,)
Shape of reshaped encoded_y_1d: (686,)
Baseline: 80.19% (6.57%)


In [17]:
estimator.fit(X_train,Y_train)
y_preds = estimator.predict(X_test)

In [18]:
y_preds


array([1, 0, 0, 1, 1, 2, 0, 1, 2, 1, 2, 2, 1, 0, 0, 3, 3, 1, 3, 2, 0, 0,
       3, 0, 1, 0, 0, 3, 3, 2, 1, 1, 0, 1, 2, 0, 2, 1, 0, 3, 1, 0, 0, 3,
       3, 3, 0, 1, 1, 0, 1, 1, 3, 0, 1, 3, 1, 1, 1, 1, 1, 0, 2, 0, 3, 1,
       2, 2, 0, 3, 1, 3, 3, 0, 0, 0, 3, 3, 1, 0, 3, 2, 1, 1, 0, 1, 3, 0,
       0, 1, 1, 0, 1, 3, 3, 1, 1, 0, 3, 1, 1, 0, 0, 3, 3, 2, 2, 0, 1, 2,
       3, 3, 0, 0, 2, 1, 3, 3, 3, 0, 3, 0, 2, 0, 1, 0, 0, 0, 0, 1, 3, 3,
       1, 1, 1, 3, 3, 2])

In [19]:
#Join the model and the scaler in a Pipeline
pip = Pipeline([('minmaxscaler',MinMaxScaler()),('keras',KerasClassifier(build_fn=base_model,epochs=300, batch_size=200,verbose=0))])
#Fit the Pipeline
pip.fit(X2,encoded_y)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('keras',
                 KerasClassifier(batch_size=200, build_fn=<function base_model at 0x000001C2927A6290>, epochs=300, verbose=0))])

In [20]:
def predict_mood(preds):
    
#     pipe = Pipeline([('minmaxscaler',MinMaxScaler()),('keras',KerasClassifier(build_fn=base_model,epochs=300, batch_size=200,verbose=0))])
#     #Fit the Pipeline
#     pipe.fit(X2,encoded_y)

    preds_features = np.array(preds[:]).reshape(-1,1).T

    #Predict the features of the song
    results = pip.predict(preds_features)

    mood = np.array(target['mood'][target['encode']==int(results)])

    return str(mood[0])
    #print(f"{name_song} by {artist} is a {mood[0].upper()} song")

In [ ]:
res = []

for i in range(len(mood_trans_np)):
  res.append(predict_mood(mood_trans_np[i]))

In [ ]:
spotify_df.shape

(156410, 25)

In [ ]:
print(len(res))

156410


In [ ]:
spotify_df['Mood'] = np.resize(res,len(spotify_df))

NameError: name 'np' is not defined

In [ ]:
spotify_df.to_csv('Spotify_Tracks_Final.csv')

NameError: name 'spotify_df' is not defined

In [ ]:
res.count("Sad")

69500

In [ ]:
res.count("Happy")

45512

In [ ]:
res.count("Energetic")

24164

In [ ]:
res.count("Calm")

17234

In [ ]:
spotify_df.shape

(156410, 26)